In [119]:
import os
import pandas as pd
import numpy as np
import json
import warnings
from sklearn.model_selection import TimeSeriesSplit
from datetime import datetime
from zipline.utils.factory import load_bars_from_yahoo
import pytz
import numpy as np
from sklearn import model_selection
from sklearn import preprocessing
import matplotlib.pyplot as plt
import ystockquote

In [19]:
symbols = ['A', 'AA', 'AAP']#, 'ABBV', 'ABT', 'ACN', 'ADBE', 'AES', 'AET', 'AFL']
start = datetime(2000, 1, 1, 0, 0, 0, 0, pytz.utc)
end = datetime(2016, 4, 30, 0, 0, 0, 0, pytz.utc)
dates = pd.date_range(start,end)

In [89]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30'],
              dtype='datetime64[ns, UTC]', length=5965, freq='D')

In [34]:
dates = pd.date_range('2011-10-13', '2016-10-12')
dates

DatetimeIndex(['2011-10-13', '2011-10-14', '2011-10-15', '2011-10-16',
               '2011-10-17', '2011-10-18', '2011-10-19', '2011-10-20',
               '2011-10-21', '2011-10-22',
               ...
               '2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
               '2016-10-07', '2016-10-08', '2016-10-09', '2016-10-10',
               '2016-10-11', '2016-10-12'],
              dtype='datetime64[ns]', length=1827, freq='D')

In [5]:
def get_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    data = []
    for symbol in symbols:
        df_temp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan'])
        df_comp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, na_values=['nan'])
        data.append(df_comp)
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        #print df_temp
        df = df.join(df_temp)
    return [df, data]

In [6]:
[df_old, data] = get_data(symbols, dates)

In [9]:
df_old.size/3

1827

In [11]:
data

[                 Open       High        Low      Close    Volume  Adj Close
 Date                                                                       
 2016-10-12  45.779999  45.820000  45.419998  45.470001   1730400  45.470001
 2016-10-11  47.189999  47.189999  45.419998  45.689999   2912500  45.689999
 2016-10-10  47.580002  48.150002  47.549999  47.689999   1544900  47.689999
 2016-10-07  47.330002  47.490002  46.990002  47.380001   1663100  47.380001
 2016-10-06  47.009998  47.430000  46.880001  47.299999   1772800  47.299999
 2016-10-05  47.040001  47.310001  46.930000  47.150002   1431900  47.150002
 2016-10-04  47.070000  47.299999  46.669998  46.830002   1764000  46.830002
 2016-10-03  46.820000  47.230000  46.820000  47.139999   1538800  47.139999
 2016-09-30  46.500000  47.320000  46.299999  47.090000   1754300  47.090000
 2016-09-29  47.049999  47.270000  46.139999  46.410000   1938200  46.294998
 2016-09-28  46.900002  47.259998  46.520000  47.180000   1502500  47.063091

In [127]:
df_old.isnull().values.any()

True

In [128]:
df_old.isnull().sum().sum()

1707

In [119]:
len(dates)

1827

In [16]:
def get_stocks_info(symbol):
    dict_stocks = ystockquote.get_historical_prices(symbol, '2011-10-13', '2016-10-12')
    json_str = json.dumps(dict_stocks)
    df =pd.read_json(json_str,orient='index')
    df = df.reindex(df.index.rename('Date'))
    return df

In [14]:
def fill_missing(df):
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

In [35]:
df = get_stocks_info('GOOGL')

In [54]:
ticker='GOOGL'

In [45]:
fill_missing(s)

In [37]:
new_df = df.filter(['Date','Adj Close'], axis=1)

In [38]:
new_df.isnull().values.any()

False

In [39]:
s = new_df.reindex(dates, fill_value=np.nan)

In [40]:
s.isnull().values.any()

True

In [43]:
s.isnull().sum().sum()*3

1707

In [61]:
s = s.reindex(s.index.rename('Date'))

In [62]:
s

,Adj Close
Date,
2011-10-13,279.774784
2011-10-14,296.136143
2011-10-15,296.136143
2011-10-16,296.136143
2011-10-17,291.496494
2011-10-18,295.550541
2011-10-19,290.640659
2011-10-20,292.127140
2011-10-21,295.540531


In [12]:
def smavg(df, N):
    return pd.rolling_mean(df, N)[N - 1:]


def expmavg(df, span):
    return pd.ewma(df, span=span)


def com_daily_returns(df):
    daily_returns = (df / df.shift(1)) - 1
    return daily_returns


def com_momentum(df, n):
    mom = (df / df.shift(n))
    return mom


def com_rsi(df, n):
    df = df.diff()
    df = df[1:]
    up, down = df.copy(), df.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = pd.rolling_mean(up, n)
    roll_down = pd.rolling_mean(down.abs(), n)
    rs = roll_up / roll_down
    rss = 100.0 - (100.0 / (1.0 + rs))
    return rss

def BBW(data,length):
    return 4*pd.stats.moments.rolling_std(data,length)

def com_williamspercent(df):
    high=pd.rolling_max(df,2)
    low=pd.rolling_min(df,2)
    return ((high-df)/(high-low))*100

In [63]:
df.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2011-10-13,279.774784,558.990018,559.000018,548.019981,550.029993,11363800
2011-10-14,296.136143,591.680013,599.600003,587.569992,599.470006,17042700
2011-10-17,291.496494,582.409995,591.830010,577.999981,583.720024,8008300
2011-10-18,295.550541,590.509981,592.559992,577.399996,580.189989,7593900
2011-10-19,290.640659,580.700037,592.060004,579.220012,587.339997,5857100


In [130]:
temp_df = s.copy()
comp_temp = df.copy()
volume = comp_temp['Volume']
high = comp_temp['High']
close = comp_temp['Close']
low = comp_temp['Low']
# print type(volume)
sum_data = temp_df.join(high)
sum_data =sum_data.join(close)
sum_data =sum_data.join(low)
sum_data =sum_data.join(volume)
fill_missing(sum_data)
sum_data = sum_data[['High','Low','Close','Volume']]
low = sum_data['Low']
high =sum_data['High']
volume = sum_data['Volume']
close = sum_data['Close']
sma10 = smavg(temp_df, 3)
print temp_df.head()
print sma10.head()
sma10 = sma10.rename(columns={'Adj Close':'sma10'})
print temp_df.head()
print sma10.head()
sma20 = smavg(temp_df, 20)
sma20 = sma20.rename(columns={'Adj Close':'sma20'})
sma30 = smavg(temp_df, 30)
sma30 = sma30.rename(columns={'Adj Close':'sma30'})
ema10 = expmavg(temp_df, 10)
ema10 = ema10.rename(columns={'Adj Close':'ema10'})
ema20 = expmavg(temp_df, 20)
ema20 = ema20.rename(columns={'Adj Close':'ema20'})
ema30 = expmavg(temp_df, 30)
ema30 = ema30.rename(columns={'Adj Close':'sma30'})
mom = com_momentum(temp_df, 10)
mom = mom.rename(columns={'Adj Close':'mom'})
drr = com_daily_returns(temp_df)
drr = drr.rename(columns={'Adj Close':'drr'})
bbw = BBW(temp_df, 20)
bbw = bbw.rename(columns={'Adj Close':'bbw'})
will = com_williamspercent(temp_df)
will = will.rename(columns={'Adj Close':'will'})
rsi = com_rsi(temp_df, 10)
rsi = rsi.rename(columns={'Adj Close':'rsi'})
rsi = rsi.dropna()
# print rsi.shape
rsi = rsi.join(drr)
rsi = rsi.join(mom)
rsi = rsi.join(bbw)
# rsi = rsi.join(will)
rsi = rsi.join(ema10)
rsi = rsi.join(ema20)
rsi = rsi.join(ema30)
rsi = rsi.join(sma10)
rsi = rsi.join(sma20)
rsi = rsi.join(sma30)
# print rsi.shape
rsi = rsi.join(volume)
rsi = rsi.join(high)
rsi = rsi.join(close)
temp_df = temp_df.shift(-n)
#print temp_df
rsi = rsi.join(temp_df)
# print rsi.shape

rsi = rsi.dropna()

             Adj Close
Date                  
2011-10-13  279.774784
2011-10-14  296.136143
2011-10-15  296.136143
2011-10-16  296.136143
2011-10-17  291.496494
             Adj Close
Date                  
2011-10-15  290.682357
2011-10-16  296.136143
2011-10-17  294.589593
2011-10-18  294.394393
2011-10-19  292.562565
             Adj Close
Date                  
2011-10-13  279.774784
2011-10-14  296.136143
2011-10-15  296.136143
2011-10-16  296.136143
2011-10-17  291.496494
                 sma10
Date                  
2011-10-15  290.682357
2011-10-16  296.136143
2011-10-17  294.589593
2011-10-18  294.394393
2011-10-19  292.562565


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=3,center=False).mean()
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=20,center=False).mean()
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=30,center=False).mean()
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: pd.ewm_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.ewm(ignor

ValueError: columns overlap but no suffix specified: Index([u'sma30'], dtype='object')

In [101]:
ti = pd.DataFrame({
        'sma10': sma10,
         'sma20': sma20,
#         'sma30': sma30,
#         'ema10': ema10,
#         'ema20': ema20,
#         'ema30': ema30,
#         'drr':drr,
#         'bbw':bbw,
#         'rsi':rsi,
#         'will':will,
#         'mom':mom,
#         'volume':volume,
#         'high':high,
#        'close':close
    },index=s.index)


ValueError: Shape of passed values is (2, 1825), indices imply (2, 1827)

In [131]:
X=pd.concat([ema10,ema20,sma10,rsi,drr,mom,will], axis=1).dropna()

In [158]:
Y=pd.concat([X.copy(),s.copy()], axis=1,join_axes=[X.index]).dropna()

In [164]:
index_values = X.index.values

In [194]:
Y = s.reindex(index_values, fill_value=np.nan)

In [195]:
Y=Y.reset_index()

In [196]:
Y=Y[Y['Date'].isin(index_values)]

In [202]:
Y = Y.set_index('Date')

In [208]:
Y = Y.reset_index(drop=True)
X=X.reset_index(drop=True)
df.head()

,Adj Close
0,289.614628
1,292.702702
2,299.049046
3,298.368382
4,304.469486


In [204]:
Y.shape

(1245, 1)

In [154]:
X.shape

(1245, 15)

In [120]:
tscv = TimeSeriesSplit(n_splits=3)

In [214]:
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_index = hash(tuple(train_index))
    X_train = X[[]]#, X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    min_error = 10000000000
    error = []
    deg = [2,3,4,5,6]
    degr = 2
    # pca = PCA(n_components=15)
    # train_data = pca.fit_transform(train_data)
    # test_data = pca.fit_transform(test_data)
    for i in range(2,7):
        poly = PolynomialFeatures(degree=i)
        train_data_ = poly.fit_transform(train_data)
        test_data_ = poly.fit_transform(test_data)
        clf = linear_model.LinearRegression()
        clf.fit(train_data_, train_label)
        predict = clf.predict(test_data_)
        mse = MSE(predict,test_label)
        error.append(mse)
        # print mse[0]
        if( mse[0] < min_error ):
            min_error = mse[0]
            degr = i
    plt.title('MSE vs Degree')
    plt.plot(deg,error)
    plt.xlabel('Degree')
    plt.ylabel('MSE')
    directory = './poly2/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.savefig(directory + 'MSE' + parameters + '.png')
    plt.close()

('TRAIN:', array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 1

TypeError: unhashable type: 'numpy.ndarray'